In [1]:
# --------------------------------------
# 📌 STEP 0: Import Important Libraries
# --------------------------------------

In [2]:
import pandas as pd
import datetime
from geopy.distance import geodesic
from collections import defaultdict
import searoute as sr
import json

In [3]:
# ----------------------------
# 📌 STEP 1: Load Port Data
# ----------------------------

In [4]:
# Load the port database (CSV file with global port info)
df_ports = pd.read_csv("C:\\Users\\Divya.M\\Downloads\\archive\\UpdatedPub150.csv")

In [5]:
# ------------------------------------
# 📌 STEP 2: Define Utility Functions
# ------------------------------------

In [6]:
# Normalize longitude to range [-180, 180]
def normalize_lon(lon):
    return (lon + 180) % 360 - 180

# Simulate vessel movement by interpolating between route coordinates
def simulate_vessel_track(route_coords, start_time=None, speed_knots=20, interval_minutes=5):
    track_points = []
    speed_kmph = speed_knots * 1.852
    interval_hours = interval_minutes / 60
    distance_per_interval = speed_kmph * interval_hours

    if start_time is None:
        start_time = datetime.datetime.now(datetime.timezone.utc)
    current_time = start_time

    for i in range(len(route_coords) - 1):
        start = route_coords[i]
        end = route_coords[i + 1]
        leg_distance_km = geodesic(start, end).km

        if leg_distance_km == 0:
            continue

        num_steps = max(1, int(leg_distance_km // distance_per_interval))

        for step in range(num_steps):
            fraction = step / num_steps
            lat = start[0] + (end[0] - start[0]) * fraction
            lon = start[1] + (end[1] - start[1]) * fraction
            lon = normalize_lon(lon)


            track_points.append({
                "timestamp": current_time.isoformat() + "Z",
                "latitude": lat,
                "longitude": lon,
                "speed": speed_knots,   
                "mmsi": vessel["mmsi"]
            })
            current_time += datetime.timedelta(minutes=interval_minutes)
        
    return track_points

In [17]:
# ----------------------------
# 📌 STEP 3: Define Vessels
# ----------------------------

In [8]:
# Each vessel has a unique MMSI, origin and destination port
# 🛳️ List of vessels with port names, MMSI, and speed
vessels = [
    {"mmsi": 123456789, "origin": "Shanghai", "destination": "Los Angeles", "speed_knots": 18},
    {"mmsi": 987654321, "origin": "Singapore", "destination": "Sydney", "speed_knots": 22},
    {"mmsi": 192837465, "origin": "New York", "destination": "Rotterdam", "speed_knots": 16}
]

# Final list to hold all simulated points
all_tracks = []

In [9]:
# -------------------------------------------
# 📌 STEP 4: Simulate Tracks for All Vessels
# -------------------------------------------

In [10]:
for vessel in vessels:
    try:
        # Step 1: Extract coordinates from port DB
        origin = df_ports[df_ports['Main Port Name'].str.contains(vessel['origin'], case=False, na=False)][['Longitude', 'Latitude']].iloc[0]
        destination = df_ports[df_ports['Main Port Name'].str.contains(vessel['destination'], case=False, na=False)][['Longitude', 'Latitude']].iloc[0]
        origin_coords = (origin['Longitude'], origin['Latitude'])
        destination_coords = (destination['Longitude'], destination['Latitude'])

        # Step 2: Get the sea route
        route = sr.searoute(origin_coords, destination_coords)
        route_coords = [(lat, lon) for lon, lat in route['geometry']['coordinates']]  # Swap to (lat, lon)

        # Step 3: Simulate movement along route
        vessel_track = simulate_vessel_track(route_coords, speed_knots=vessel["speed_knots"])

        # Step 4: Attach MMSI to each point
        for point in vessel_track:
            point["mmsi"] = vessel["mmsi"]

        # Step 5: Add to full track list
        all_tracks.extend(vessel_track)

        print(f"✅ Track generated for {vessel['origin']} → {vessel['destination']} (MMSI {vessel['mmsi']})")

    except Exception as e:
        print(f"⚠️ Error with {vessel['origin']} → {vessel['destination']}: {e}")

print(f"\n🎉 Track generation complete! Total points: {len(all_tracks)}")

✅ Track generated for Shanghai → Los Angeles (MMSI 123456789)
✅ Track generated for Singapore → Sydney (MMSI 987654321)
✅ Track generated for New York → Rotterdam (MMSI 192837465)

🎉 Track generation complete! Total points: 11487


In [11]:
# --------------------------------
# 📌 STEP 5: Validate Coordinates
# --------------------------------

In [12]:
validated_tracks = []
invalid_points = 0
valid_points_by_mmsi = defaultdict(int)

for point in all_tracks:
    lat = point['latitude']
    lon = point['longitude']
    mmsi = point['mmsi']

    if -90 <= lat <= 90 and -180 <= lon <= 180:
        validated_tracks.append(point)
        valid_points_by_mmsi[mmsi] += 1
    else:
        invalid_points += 1

print(f"\n🛡️ Validation complete: {len(validated_tracks)} valid points, {invalid_points} invalid discarded.")


🛡️ Validation complete: 11487 valid points, 0 invalid discarded.


In [13]:
# -------------------------
# 📌 STEP 6: Sample Output
# -------------------------

In [14]:
tracks_by_mmsi = defaultdict(list)
for point in validated_tracks:
    tracks_by_mmsi[point['mmsi']].append(point)

# Print sample for each vessel
for mmsi, points in tracks_by_mmsi.items():
    print(f"\n🚢 MMSI {mmsi}: First 5 points:")
    for p in points[:5]:
        print(p)

# Print total valid points per vessel
print("\n📋 Valid points per vessel:")
for mmsi, count in valid_points_by_mmsi.items():
    print(f"🚢 MMSI {mmsi}: {count} points")


🚢 MMSI 123456789: First 5 points:
{'timestamp': '2025-04-30T10:13:52.788336+00:00Z', 'latitude': 30.732393, 'longitude': 121.82739300000003, 'speed': 18, 'mmsi': 123456789}
{'timestamp': '2025-04-30T10:18:52.788336+00:00Z', 'latitude': 30.73591393103448, 'longitude': 121.85671048275861, 'speed': 18, 'mmsi': 123456789}
{'timestamp': '2025-04-30T10:23:52.788336+00:00Z', 'latitude': 30.739434862068965, 'longitude': 121.88602796551726, 'speed': 18, 'mmsi': 123456789}
{'timestamp': '2025-04-30T10:28:52.788336+00:00Z', 'latitude': 30.742955793103448, 'longitude': 121.91534544827584, 'speed': 18, 'mmsi': 123456789}
{'timestamp': '2025-04-30T10:33:52.788336+00:00Z', 'latitude': 30.746476724137928, 'longitude': 121.94466293103449, 'speed': 18, 'mmsi': 123456789}

🚢 MMSI 987654321: First 5 points:
{'timestamp': '2025-04-30T10:13:52.964920+00:00Z', 'latitude': 1.1, 'longitude': 103.60000000000002, 'speed': 22, 'mmsi': 987654321}
{'timestamp': '2025-04-30T10:18:52.964920+00:00Z', 'latitude': 1.11

In [15]:
# -----------------------------------
# 📌 STEP 7: (Optional) Save to File
# -----------------------------------

In [16]:
# Save to JSON
with open("multi_vessel_tracks.json", "w") as f:
    json.dump(validated_tracks, f, indent=2)

print("\n💾 Saved validated tracks to 'multi_vessel_tracks.json'")


💾 Saved validated tracks to 'multi_vessel_tracks.json'
